In [1]:
!pwd
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124\run_objective_3l


'c:\\users\\81908\\appdata\\local\\pypoetry\\cache\\virtualenvs\\tfgpu-ehdmne1y-py3.8\\scripts\\python.exe'

In [2]:
# tensorflow2.0 + kerasでGPUメモリの使用量を抑える方法(最小限だけ使うように設定)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
import sys
sys.path.append(r'C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124')
import moa_MLPs_funcs

Tensorflow version 2.3.1
n_classes: 206
DEBUG


In [4]:
import optuna


def objective(trial):
    model_type = "3l_v2"
    params = {}
    params["activation"] = trial.suggest_categorical("activation", ["relu", "elu", "selu"])
    params["denses"] = trial.suggest_categorical(
        "denses",
        [
            [512, 448, 384],
            [1024, 896, 768],
            [2048, 1792, 1536],
            [2560, 2048, 1524],
            [512, 512, 512],
            [1024, 1024, 1024],
            [1024, 768, 1536],
            [1024, 1536, 768],
            [512, 768, 1536],
            [1159, 960, 1811],
            [1159, 1811, 960],
            [960, 1159, 1811],
        ],
    )
    drop_rates = []
    for i in range(4):
        drop_rate = trial.suggest_uniform(f'drop_rate{i}', 0.1, 0.9)
        drop_rates.append(drop_rate)
    params["drop_rates"] = drop_rates
    print("-" * 100)
    print(f"params: {params}")

    oof_score, Y_pred = moa_MLPs_funcs.train_and_evaluate(model_type=model_type, params=params)

    return oof_score

In [5]:
%%time

n_trials = 150
#n_trials = 50
#n_trials = 3

study = optuna.create_study(
    study_name="study",
    storage=f"sqlite:///study.db",
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=1),
)
study.optimize(objective, n_trials=n_trials)
study.trials_dataframe().to_csv(f"objective_history.csv", index=False)
with open(f"objective_best_params.txt", mode="w") as f:
    f.write(str(study.best_params))
print(f"\nstudy.best_params:\n{study.best_params}")

[I 2020-11-25 20:45:13,027] A new study created in memory with name: study
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 448, 384] which is of type list.
  warnings.warn(message)
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1024, 896, 768] which is of type list.
  warnings.warn(message)
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\optuna\distributions.py:404: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [204

----------------------------------------------------------------------------------------------------
params: {'activation': 'elu', 'denses': [960, 1159, 1811], 'drop_rates': [0.6762595947537264, 0.10009149985387591, 0.3418660581054718, 0.21740471265369044]}



c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/2
12/12 [==============================] - 3s 229ms/step - loss: 0.7410 - binary_crossentropy: 0.7410 - val_loss: 0.6762 - val_binary_crossentropy: 0.6761
Epoch 2/2
12/12 [==============================] - 2s 181ms/step - loss: 0.5891 - binary_crossentropy: 0.5889 - val_loss: 0.3925 - val_binary_crossentropy: 0.3917
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-------------

c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


...

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/2
12/12 [==============================] - 3s 230ms/step - loss: 0.7480 - binary_crossentropy: 0.7480 - val_loss: 0.6756 - val_binary_crossentropy: 0.6756
Epoch 2/2
12/12 [==============================] - 2s 167ms/step - loss: 0.5876 - binary_crossentropy: 0.5874 - val_loss: 0.4520 - val_binary_crossentropy: 0.4513
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-------------

c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/2
12/12 [==============================] - 3s 215ms/step - loss: 0.7465 - binary_crossentropy: 0.7465 - val_loss: 0.6765 - val_binary_crossentropy: 0.6764
Epoch 2/2
12/12 [==============================] - 2s 163ms/step - loss: 0.5862 - binary_crossentropy: 0.5859 - val_loss: 0.6682 - val_binary_crossentropy: 0.6677
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-------------

[I 2020-11-25 20:47:39,282] Trial 2 finished with value: 0.37079594068579247 and parameters: {'activation': 'selu', 'denses': [1024, 1024, 1024], 'drop_rate0': 0.6481756003174076, 'drop_rate1': 0.263561799785214, 'drop_rate2': 0.8024939491127564, 'drop_rate3': 0.12191007455834094}. Best is trial 0 with value: 0.32903415833486926.


oof_score: 0.37079594068579247
----------------------------------------------------------------------------------------------------

study.best_params:
{'activation': 'elu', 'denses': [960, 1159, 1811], 'drop_rate0': 0.6762595947537264, 'drop_rate1': 0.10009149985387591, 'drop_rate2': 0.3418660581054718, 'drop_rate3': 0.21740471265369044}
Wall time: 2min 26s
